In [ ]:
# Importing relevant packages

import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time
import matplotlib.pyplot as plt
import bs4 as bs
import pickle
import requests

In [ ]:
# Getting all SP500 tickers:

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

tickers = save_sp500_tickers()
tickers = [x[:-1] for x in tickers]

#Replacing tickername that creates an error later.
tickers = [w.replace('BF.B', 'BF-B') for w in tickers]
print(tickers)

In [ ]:
# Finding all stock information

ETFs = np.array(tickers) # all ETFs used

api_key = 'IHXRIDZAXOXUHF8T' # AlphaVantage API key

ts = TimeSeries(key = api_key, output_format = 'pandas', indexing_type='date')

print(ETFs)

In [ ]:
price_data = pd.DataFrame()
# Loop through requested securities
print('Querying Securities, Estimated time: ' + str(round(len(ETFs)/5)) + ' minutes')
for x in range(len(ETFs)):
    print(str(ETFs[x]))

    if (x + 1) % 5 == 0:
        time.sleep(60) # wait 1 minutes after every 5 API calls

    data, meta_data = ts.get_monthly_adjusted(symbol=str(ETFs[x])) 
    data = data['5. adjusted close'].iloc[::-1]             
    data = pd.DataFrame(data).rename(index=str, columns={'5. adjusted close' : str(ETFs[x])})
    price_data = pd.concat([price_data,data], axis=1, sort=False)
    
price_data = price_data.iloc[:300].iloc[::-1]

plt.plot(price_data)
plt.xticks([])

In [ ]:
localdata = price_data.copy()

localdata.to_csv(r'C:\Users\07anl_000\Desktop\Polit - KU\Kandidat\3. semester\Seminar Anv. Corp Fin\localdata.csv')